# Vamos a hacer un censo de piscinas

¿Cómo podemos hacer esto?

Podemos contratar a 100 personas y que durante una semana recorran todos los hogares de tu municipio para que pregunten, casa por casa:

* ¿Tienes una piscina?
* En caso afirmativo, ¿de qué tamaño?
* ...

O, podemos coger imágenes satelitales u ortofotos y, mediante un algoritmo, hacer un recuento.

Veamos un caso práctico. En Baleares tenemos un IDE (Infraestructura de Datos Espaciales, http://ideib.caib.es/visualitzador/visor.jsp) que incluye ortofotos a resolución de 0.25m (cada pixel corresponde a 25 cm).

Podemos obtener la información mediante peticiones a un WMS (Web Map Service). Por ejemplo:

http://ideib.caib.es/pub_ideib/public/Ortofoto/MapServer/WMSServer?SERVICE=WMS&LAYERS=0%2C1%2C2&VERSION=1.3.0&REQUEST=GetMap&STYLES=%2C%2C&EXCEPTIONS=INIMAGE&FORMAT=image%2Fjpeg&TRANSPARENT=FALSE&CRS=EPSG%3A25831&BBOX=470000,4380000,470500,4380500&WIDTH=2000&HEIGHT=2000

El WMS, simplificando, es un servidor que permite que le pasemos determinados parámetros y nos devuelve lo que le pedimos. En este caso, en la URL anterior vemos que tenemos `CRS`, `BBOX`, `WIDTH`, `HEIGHT`. Vamos a pedir una serie de imágenes cambiando los parámetros del `BBOX`:

In [ ]:
from pathlib import Path
from urllib import request

url = (
    "http://ideib.caib.es/pub_ideib/public/Ortofoto/MapServer/WMSServer?SERVICE=WMS&"
    "LAYERS=0%2C1%2C2&VERSION=1.3.0&REQUEST=GetMap&STYLES=%2C%2C&"
    "EXCEPTIONS=INIMAGE&FORMAT=image%2Fjpeg&TRANSPARENT=FALSE&CRS=EPSG%3A25831&"
    "BBOX={x0},{y0},{x1},{y1}&"
    "WIDTH=2000&HEIGHT=2000"
)
filename = "{x0}_{y0}_500m_2000x2000.jpeg"
pathfile = Path(".", "piscinas", "imgs")
for x in range(465000, 470000, 500):
    for y in range(4378000, 4383000, 500):
        request.urlretrieve(
            url.format(x0=x, y0=y, x1=x + 500, y1=y + 500),
            filename= pathfile / filename.format(x0=x, y0=y)
        )

Acabamos de descargar 100 imágenes de 2000x2000 píxeles, cada una con tres canales (R, G, B) con valores de 1 byte (8 bits, UINT8). Si queremos leerlo todo en memoria necesitamos alrededor de:

$$2000\,x\,2000\,x\,100\,x\,3\,x\,8\,=\,9600\,000\,000\,bits$$

$$2000\,x\,2000\,x\,100\,x\,3\,x\,1\,=\,1200\,000\,000\,bytes\,(~1100\,Mb)$$

Para un área de 5 km x 5 km.

En caso de que no todo el mundo tenga mucha RAM vamos a hacer solo un trozo.

In [ ]:
from pathlib import Path
import numpy as np
import sys
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
data = np.zeros((4000, 4000, 3), dtype=np.uint8)

In [ ]:
sys.getsizeof(data) / 1024**2

4 imágenes ocuparían en memoria poco más de 45Mb. Vamos a trabajar solo con una.

Vamos a leer una única imagen y a guardarla en una única estructura de datos, `data`:

In [ ]:
filename = "465000_4378000_500m_2000x2000.jpeg"
pathfile = Path(".", "piscinas", "imgs", filename)
data = plt.imread(pathfile)
        
plt.imshow(data)
plt.show()

In [ ]:
pool_detector = (
    (data[:,:,0] > 100) & (data[:,:,0] < 150) & 
    (data[:,:,1] > 170) & (data[:,:,1] < 210) & 
    (data[:,:,2] > 190) & (data[:,:,2] < 220)
)
plt.imshow(pool_detector, cmap=plt.cool())

In [ ]:
plt.imshow(data)
plt.imshow(pool_detector, alpha=0.75, cmap=plt.cool())

¿Cuánta agua puede suponer esto?

In [ ]:
pool_detector.sum() / 16 # m2

Pensemos que la profundidad media son 1.5 metros y que 1 $m^{3}$ son 1000 litros

In [ ]:
pool_detector.sum() * 1500 / 16 # litros

Algunos enlaces interesantes con datos y formas de analizarlos:
* http://rexdouglass.com/quick-and-dirty-land-cover-estimates-from-landsat-satellite-imagery-and-openstreetmap/
* https://github.com/Fernerkundung/awesome-sentinel/
* https://earthengine.google.com/datasets/
* https://aws.amazon.com/es/public-datasets/
* ...